<a href="https://colab.research.google.com/github/iampundir/Genetic-algorithm/blob/master/Shubham_pundir_Simple_Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding max probability of Normal Distribution using GA

Equation of a normal distribution is $$p(x) = \frac {1}{\sqrt{2 \pi \sigma^2}}exp \left [- \frac{||x-\mu ||^2}{2\sigma^2} \right ]$$ which attains maximum value at the mean $\mu$. 

<img src="https://www.intmath.com/counting-probability/img/snorm2.gif"/> 

In this example, we would use Genetic Algorithm to find the value of $x$ for which the function value (i.e. the probablity) is maximum. For this purpose, $\mu = 100$ and $\sigma = 5$ are considered

In [ ]:
import numpy as np

In [ ]:
# Define the value of function output. Here mu = 100 and sigma = 5

def f(x):
    return (1/np.sqrt(2*np.pi*25))*np.exp(-((x-100)**2)/(2*25))

In [ ]:
# We need initial population of chromosomes. Let us create 10 such chromosomes. This is the initialization step
# Each chromosome is 5 bits long and each one corresponds to some integer value
# np.random.seed(123) # Just to maintain consistency in output of randome number generation
population = np.array(np.random.binomial(1,0.3,size=(10,10)))
population

array([[0, 1, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
# Convert bit sequence to integer
# If the chromosomes were created as strings (e.g. '10110'), inbuilt int() function could be used conveniently
# Try that method on your own

def bit_to_int(bit_list):
    n = len(bit_list)
#     print(n)
    val = 0
    for i in range(n):
        val += bit_list[i]*2**(n-i-1)
#         print(val)
    return val

In [ ]:
bit_to_int(population[0,:])

268

In [ ]:
# The next task is to calculate the function value for all the chromosomes

F = [f(bit_to_int(y)) for y in population]

# Afterward, convert the function values to probabilities

F_p = F/sum(F)
F_p

array([1.25955996e-244, 9.99999729e-001, 0.00000000e+000, 2.71310946e-007,
       1.31936102e-084, 0.00000000e+000, 1.41995408e-213, 9.71043382e-067,
       0.00000000e+000, 7.42946871e-211])

In [ ]:
# Choose two chromosomes randomly based on the probability distribution
index = np.random.choice(range(len(population)), size=2, p=F_p)
index

array([1, 1])

In [ ]:
# Perform crossover with randomly chosen crossover point with some crossover probability

pc = 0.75 # Crossover probability

#Crossover point should be lying between the first and the last bit
index_crossover = np.random.choice(range(1,population.shape[1]-1), size=1)[0]
print(index_crossover)

# Create 2 new chomosomes using crossover
if(np.random.uniform(size=1)[0]<pc):
    ch_new_1 = np.concatenate([population[index[0],:index_crossover],population[index[1],index_crossover:]])
    ch_new_2 = np.concatenate([population[index[1],:index_crossover],population[index[0],index_crossover:]])
else:
    ch_new_1 = population[index[0],:]
    ch_new_2 = population[index[1],:]

print(ch_new_1)
print(ch_new_2)

8
[0 0 0 1 0 1 1 0 0 0]
[0 0 0 1 0 1 1 0 0 0]


In [ ]:
# Apply mutation on those two newly created chromosomes at each bit with some mutation prpbability
pm = 0.005
for i in range(len(ch_new_1)):
    if (np.random.uniform(size=1)[0]<pm):
        ch_new_1[i] = np.where(ch_new_1[i]==1,0,1)

for i in range(len(ch_new_2)):
    if (np.random.uniform(size=1)[0]<pm):
        ch_new_2[i] = np.where(ch_new_2[i]==1,0,1)
        
print(ch_new_1)
print(ch_new_2)

[0 0 0 1 0 1 1 0 0 1]
[0 0 0 1 0 1 1 0 0 1]


In [ ]:
# Two new chromosomes are available. We need 8 more chromosomes. 
# Hence the above process is to be repeated 4 more times

# We can put everything in a loop inside a function
def simple_ga_generation(pop, func, maximize= True, pc = 0.75, pm = 0.005, eps=1e-5):
    pc = pc
    pm = pm
    
    population = pop
    
    if maximize == maximize:
        f = func
    else:
        f = 1/(func + eps)

    if population.shape[0] % 2 ==0:
        n_iter = int(population.shape[0]/2)
    else:
        n_iter = population.shape[0]//2 + 1
#     print (n_iter)

    pop_old = population.copy()
    pop_new = np.zeros(shape=population.shape)

    for iteration in range(n_iter):
        F = [f(bit_to_int(y)) for y in pop_old]
        F_p = F/sum(F)
    
        index = np.random.choice(range(len(pop_old)), size=2, p=F_p)
    
        index_crossover = np.random.choice(range(1,pop_old.shape[1]-1), size=1)[0]
    
        if(np.random.uniform(size=1)[0]<pc):
            ch_new_1 = np.concatenate([pop_old[index[0],:index_crossover],pop_old[index[1],index_crossover:]])
            ch_new_2 = np.concatenate([pop_old[index[1],:index_crossover],pop_old[index[0],index_crossover:]])
        else:
            ch_new_1 = pop_old[index[0],:]
            ch_new_2 = pop_old[index[1],:]
    
        for i in range(len(ch_new_1)):
            if (np.random.uniform(size=1)[0]<pm):
                ch_new_1[i] = np.where(ch_new_1[i]==1,0,1)
            
        for i in range(len(ch_new_2)):
            if (np.random.uniform(size=1)[0]<pm):
                ch_new_2[i] = np.where(ch_new_2[i]==1,0,1)
            
        pop_new[2*iteration,:] = ch_new_1
        pop_new[2*iteration + 1,:] = ch_new_2

    return pop_new

In [ ]:
simple_ga_generation(pop=population, func=f)

array([[0., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
       [1., 0., 0., 1., 0., 1., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 1., 0., 0., 1.]])

In [ ]:
def simple_ga(pop, func, maximize= True, pc = 0.75, pm = 0.005, max_iter=100, delta=0.000001, eps=1e-5):
    if maximize == maximize:
        f = func
    else:
        f = 1/(func + eps)
    
    
    f_old = np.array([f(bit_to_int(x)) for x in pop])
    f_max_old = max(f_old)
    print(f_max_old)
    best_ch = pop[np.argmax(f_old),:]
    
    pop_old = pop
    
    for iterations in range(max_iter):
        pop_new = simple_ga_generation(pop_old,func,maximize=maximize, pc=pc, pm=pm, eps=eps)
        f_new = np.array([f(bit_to_int(x)) for x in pop_new])
        f_max_new = max(f_new)
#         print(f_max_new)
        d = np.abs(f_max_new - f_max_old)
#         print(d)
        if (iterations>10)&(d < delta):
            return {'Max_function_value': f_max_new, 
                    'Best_chromosome': pop_new[np.argmax(f_new),:], 
                    'No_of_iterations': iterations, 
                    'Param_value': bit_to_int(pop_new[np.argmax(f_new),:])}
                    
        if f_max_new > f_max_old:
            f_max_old = f_max_new
            best_ch = pop_new[np.argmax(f_new),:]
        pop_old = pop_new.copy()
        
    print("The value didn't converge after {} iterations".format(max_iter))
    return [f_max_old, best_ch]                    

In [ ]:
population1 = np.array(np.random.binomial(1,0.3,size=(50,10)))
simple_ga(population1, f)

0.07365402806066466


{'Best_chromosome': array([0., 0., 0., 1., 1., 0., 0., 1., 0., 0.]),
 'Max_function_value': 0.07978845608028654,
 'No_of_iterations': 11,
 'Param_value': 100.0}

In [ ]:
2**6

64